# Styling of annotations

:::{note}
What you see in this notebook will depend on whether you've run this notebook before and written annotations to the `annotations.db` database! For reproducibility, the rest of the notebook will assume the `annotations.db` has been deleted (if it exists).
:::

## Setup
The first thing we need to do is get and plot the data we want to annotate. This is done by using [pandas](https://pandas.pydata.org/) and [hvplot](https://hvplot.holoviz.org/). We'll use the same data as in the [basics](Basics.ipynb) notebook.


In [ ]:
import hvplot.pandas
import pandas as pd

speed_data = pd.read_parquet("../assets/example.parquet")
curve = speed_data.hvplot("TIME", "SPEED")

The next step is to create an annotator and add some annotations to it with `define_annotations`. 

In [ ]:
from holonote.annotate import Annotator, SQLiteDB

annotator = Annotator(
    curve,
    fields=["category"],
    connector=SQLiteDB(table_name="styling"),
)

start_time = pd.date_range("2022-06-04", "2022-06-22", periods=5)
end_time = start_time + pd.Timedelta(days=2)
data = {
    "start_time": start_time,
    "end_time": end_time,
    "category": ["A", "B", "A", "C", "B"],
}
annotator.define_annotations(pd.DataFrame(data), TIME=("start_time", "end_time"))
annotator.df

To apply the annotation over the curve, you simply multiply the curve by the annotation. 

In [ ]:
annotator * curve

## Style accessor

Styling in this context is achieved through the `style` accessor of annotators, accessed using `annotator.style`. This accessor provides various methods for customizing the appearance of annotations, with two key methods being `color` and `alpha`. We will discuss these in more detail shortly.

There are three distinct style categories: `indicator`, `selected`, and `editor.` The `indicator` style is the default appearance, and in the example given above, it's represented by the color red. The `selected` style is applied when an annotation is actively selected, as indicated by the highlighting effect when you click on an annotation. Lastly, the `editor` style is used when an annotation is being edited, and this is evident when you click hold and drag the mouse anywhere on the plot, resulting in a blue region.

To modify the `color` and `alpha` properties of these three style categories, you can easily do so using the `color` and `alpha` methods provided by the style accessor. For the `selected` style, you can use the `selection_color` and `selection_alpha` settings, and for the `editor` style, the corresponding properties are `edit_color` and `edit_alpha`.

When a style changes, the plot is automatically updated, so you can easily experiment with different styles and see the results immediately. 

Let's see how this works in practice. Let's try updating the `color` and `alpha` properties of the `indicator` style.

In [ ]:
annotator.style.alpha = 0.5
annotator.style.color = "green"

If we want to update the `selected` style, we can use the `selection_color` and `selection_alpha` methods. With these changes, any selected annotation will change its color to blue and remove its transparency.

In [ ]:
annotator.style.selection_alpha = 1
annotator.style.selection_color = "blue"

Lastly, we can change the `editor` style. Note this will first show up on the following editable annotation you create.

In [ ]:
annotator.style.edit_alpha = 0.2
annotator.style.edit_color = "yellow"

If you want to reset the style to the default, you can use the `reset` method.

In [ ]:
annotator.style.reset()

## More granular styling

If more granular style customization is needed than `color` and `alpha`, you can change the `annotator.style.opts` for the indicator and selected styles and `edit_opts` for the editor style. 

In [ ]:
annotator.style.edit_opts = {"line_width": 10}

Lastly, you can use [dim transforms](https://holoviews.org/user_guide/Style_Mapping.html#what-are-dim-transforms) to make more advanced customization. An example of a dim transforms could be coloring based on the category. 

In [ ]:
import holoviews as hv

color_dim = hv.dim("category").categorize(
    categories={"A": "blue", "B": "red", "C": "green"}, default="grey"
)

annotator.style.color = color_dim

## Groupby a field
Suppose one of the fields in the annotator contains categorial information and has a limited set of options. In that case, a way to group the annotations by this field is to use the `groupby` Parameter to group by that field and the `visible` Parameter to show one of the categories. Let's group by the `category` field and show the `A` category. For convenience, we show the annotated plot again below.

In [ ]:
annotator.groupby = "category"
annotator.visible = ["B"]
annotator * curve

To get more interactivity a [Panel](https://panel.holoviz.org/) widget can be used to change the `visible` Parameter.

In [ ]:
import panel as pn

widget = pn.widgets.MultiSelect(value=["A"], options=["A", "B", "C"])
annotator.visible = widget

widget